In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import csv


### 1. Data collection
## 1.1. Get the list of places

In [ ]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


In [4]:
f = open('total_link.txt',"r")
total_link = f.read().split("\n")

## 1.2. Crawl places

In [ ]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'./all_Pages'
    s = requests.Session()

    for x in tqdm(range(start, end)):
        if count_link%10 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            #directory = f'folder_{count_page}'
            path = os.path.join(parent_dir, f"folder_{count_page}")
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./all_Pages/{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

In [ ]:
downloadPage(275, 295, total_link)

## 1.3 Parse downloaded pages

In [8]:
os.mkdir("TSV Files")
for x in os.listdir("all_Pages"):
    for y in os.listdir(f'all_Pages/{x}'):
        with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
            p = f.read()
            soup =  bs(p)
            pageAttribute = []
            placeName = findPlaceName(soup)
            placeTags = findPlaceTags(soup)
            numPeopleVisited = findNumPeopleVisited(soup)
            numPeopleWant = findNumPeopleWant(soup)
            placeDesc = findDescription(soup)
            placeShortDesc = findShortDescription(soup)
            placeNearby = findNearbyPlaces(soup)
            placeAddress = findAddress(soup)
            placeAlt, placeLong = findCordinates(soup)
            placeEditors = findPostEditors(soup)
            placePubDate = findPublishingDate(soup)
            placeRelatedList = findPlaceNear(soup)
            placeRelatedPlaces = findRelatedPlaces(soup)
            placeURL = findPageURL(soup)
            with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                tsv_writer = csv.writer(fp, delimiter='\t')
                tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
        

FileExistsError: [WinError 183] Impossibile creare un file, se il file esiste già: 'TSV Files'

In [7]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"})
    if placeName != None:
        placeName = placeName.text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[0].text)
    return peopleVisited

def findNumPeopleWant(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})[1]
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[1].text)
    return peopleVisited

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"})
    if shortDescription != None:
        shortDescription = shortDescription.text.replace("\n", "")
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    if nearbyPlaces != None:
        for place in nearbyPlaces:
            p = place.text.replace("\n","")
            nearPlaces.append(p)
        #Convert the list to set, and then back again to list to remove repetition
    return list(set(nearPlaces))

def findAddress(soup):
    strings = []
    adress_strings = soup.find("div", {"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"}).find("aside", {"class": "DDPageSiderail__details"}).find("address", {"class": "DDPageSiderail__address"}).find("div")
    for info in adress_strings:
        s = info.text.replace("\n", "")
        if s != "":
            strings.append(s)
    return " ".join(strings[:3])

def findCordinates(soup):
    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if cordinates != None:
        placeAlt, placeLong = cordinates.text.replace("\n", "").replace(" ","").split(",")
    return placeAlt, placeLong

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    s = dateString.text.replace("\n", "")
    #Let's modify it for the right format of datetime
    split = s.split()
    #Let's convert the string Month into the corrispondent number by using "strptime()" 
    split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
    #My format
    format = "%m %d, %Y"
    #Convert from String to datetime
    date = datetime.datetime.strptime(" ".join(split), format)
    return date

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



In [ ]:
import bs4
from sys import getsizeof
for x in os.listdir("all_Pages"):
    for y in os.listdir(f"all_Pages/{x}"):
        with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
                    p = f.read()
                    soup =  bs(p)
                    near = findPlaceNear(soup)
                    location_tag = soup.find_all('div', {'class':'DDPage__header-place-location'})[0].contents[0]
                    if type(location_tag) == bs4.element.NavigableString:
                        location = location_tag.text
                        print("first: " , location )
                    else:
                        location = location_tag.contents[0]
                        print(location)
                        print("Type: ", type(location))
                        print("Size: " , getsizeof(location) ,"\n")

                        location_with_text = location_tag.text
                        print(location_with_text)
                        print("Type: ", type(location_with_text))
                        print("Size: ", getsizeof(location_with_text), "\n")

                        normal_string_location = "Manhattan, New York"
                        print(normal_string_location)
                        print("Type: ", type(normal_string_location))
                        print("Size of normal string with the same content: ", getsizeof(normal_string_location))



        break
    break

In [ ]:
import bs4
bs4.element.NavigableString